# The Windows PE file format
There are tons of Python libraries for dissecting programs, one of the best is [lief](https://github.com/lief-project/LIEF).
The latter is also used inside `secml-malware` to perturb samples, as shown in the other tutorials.
Opening an executable is straight-forward:

In [1]:
pip install setuptools --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.9/807.9 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [2]:
pip install lief

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 44.2 MB/s eta 0:00:00


In [3]:
import lief

exe_path = 'PATH TO YOUR EXE'
exe_object: lief.PE = lief.parse(exe_path)

Now, the `exe_object` contains all the information of the loaded program.
We can look for all the components. For instance, here is how you can read the header metadata:

In [4]:
print('DOS Header')
print(exe_object.dos_header)

print('PE Header')
print(exe_object.header)

print('Optional Header')
print(exe_object.optional_header)

print('Sections')
for s in exe_object.sections:
	print(s.name, s.characteristics_lists)

DOS Header


AttributeError: ignored

This library is also very useful for manipulating the EXEs.
For instance, in few lines of code you can add sections to a program.

In [5]:
# Name your new section. Size constraint: up to 8 bytes at maximum!
new_section : lief.PE.Section = lief.PE.Section()
new_section.name = '.newsec'
new_section.content = [ord(i) for i in "This is my newly created section"]
new_section.characteristics = lief.PE.SECTION_CHARACTERISTICS.MEM_DISCARDABLE
exe_object.add_section(new_section)

# New section in place! Now we use lief to rebuild the binary.
builder = lief.PE.Builder(exe_object)
builder.build()
exe_object = lief.PE.parse(builder.get_build())
print('Sections')
for s in exe_object.sections:
	print(s.name, s.characteristics_lists)
builder.write('new_exe.file')

AttributeError: ignored

As you can see, the new section appeared as last one.
More information on how to use lief on the [documentation of the library](https://lief-project.github.io/doc/stable/index.html).